<h1><b>Módulo 9: Procesamiento de lenguaje natural o minería de textos</b></h1>

<h2><b><u>Tarea 4</u></b></h2>

<b>Participantes:</b><br>
* Marco Aurelio León Velarde
* Nahúm Xicohtencatl Hernandez
* Ángel Andrés Moreno Sánchez
* Carlos Eduardo Guerrero Estrella

<b>Objetivo:</b> El participante identificará el conjunto de características textuales que permiten mejorar los modelos de aprendizaje supervisado para la clasificación de textos, a partir de los métodos existentes para ello y con la ayuda de las bibliotecas implementadas en Python.

<b>Instrucciones:</b>

El participante debe con esta actividad integrar todos los conocimientos adquiridos hasta el momento, y puede incorporar los próximos conocimientos que adquirirá durante el módulo. Debe de aplicar tareas para el preprocesamiento de textos, hacer uso de expresiones regulares, incorporar características al clasificador, aplicar los algoritmos de clasificación supervisado binaria, realizar y anotar las diferentes pruebas realizadas así como los valores de F1 como métrica de evaluación.

<b>Situación a resolver:</b>

El discurso de odio se define comúnmente como cualquier comunicación que menosprecia a una persona o un grupo en función de algunas características. En el año 2019 se celebró la competencia: <i>SemEval-2019 International Workshop on Semantic Evaluation</i> ( https://alt.qcri.org/semeval2019/ ) planteándose 12 tareas. De la Task 5: <i>“Multilingual detection of hate speech against immigrants and women in Twitter (hatEval)”</i> se planteó lo referente al discurso de odio en redes sociales, en específico la red social Twitter (https://competitions.codalab.org/competitions/19935 ).

* Trabajar con la tarea A, dejando a libre escoger uno de los 2 idiomas.
* Realice diferentes pruebas (mínimo 3). Anote los resultados obtenidos por cada una de ellas, y asuma diferentes características en el entrenamiento del clasificador binario.

<b>Sugerencias:</b>

<i>Para el preprocesamiento de los textos puede:</i>

* Estandarizar el texto a minúsculas
* Eliminar las menciones a usuarios (@user)
* Eliminar las url’s
* Eliminar los emojis
* Las abreviaturas, contracciones y slangs sustituirlas por el texto equivalente
* Eliminar palabras funcionales
* Verificar si existen cifras numéricas, las cuales pueden ser reemplazadas por algún término o eliminarlas
* Tratamiento con los hashtags
* Eliminar caracteres raros y especiales
* Eliminar signos de puntuación
* Estandarizar las secuencias de varios espacios en blanco, tabuladores y saltos de línea
* Entre otras…

<i>Posibles características para tenerse en cuenta:</i>

* N-gramas de caracteres
* N-gramas de palabras
* N-gramas de etiquetas POS
* N-gramas de saltos de palabras (skip-gram)
* N-gramas de palabras funcionales
* N-gramas de símbolos de puntuación
* Entre otras…


In [1]:
import re
import pandas as pd
import numpy as np
import nltk
from nltk import *
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler

# Leemos los datos

In [2]:
corpus_train_esA = pd.read_csv('public_development_en_TaskA/train_en.tsv',delimiter='\t',encoding='utf-8')
corpus_dev_esA = pd.read_csv('public_development_en_TaskA/dev_en.tsv',delimiter='\t',encoding='utf-8')

### Limpieza en los datos
* Cambiar todas las palabras de mayúsculas a minúsculas
* Se han eliminado las '@' de @USUARIO con el fin de facilitar el etiquetado morfológico
* Quitar los links 
* Quitar los emojis
* Eliminar las stopwords
* Se han reemplazado todos los números por el símbolo '0'
* Quitar los signos de puntuación y quitar espacios (tabuladores, etc)
* Utilizar los diccionarios para cambiar los slangs, contracciones y abreviaturas

In [3]:
pattern_URL="(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})"

def procesar(file, namefile):    
    file[file.columns[1]] = [clean_text(i) for i in file[file.columns[1]]]    
    file.to_csv(namefile, sep='\t', encoding='utf-8', index=False)
    return file
    
def clean_text(text):
    text = text.lower()   
    #text=re.sub("@([A-Za-z0-9_]{1,15})", "@USUARIO", text)
    text=re.sub("@([A-Za-z0-9_]{1,15})", " ", text)
    text=re.sub(pattern_URL, " ", text)
    
    text= remove_emoji(text)
    text= remove_stopwords(text)
    text=re.sub("\d+", "0", text)
    # text=re.sub("\d+", " ", text)
    
    text=re.sub(r" +", " ", re.sub(r"\t", " ", re.sub(r"\n+", "\n", re.sub('(?:[.,\/!$%?¿?!¡\^&\*;:{}=><\-_`~()”“"\'\|])', " ",text))))
    text = text.strip()
    return text

def remove_stopwords(text):    
    stopwords=set(nltk.corpus.stopwords.words("english"))
    for i in stopwords:
        text = re.sub(r"\b%s\b" % i, " ", text)
    return text

def remove_emoji(text):
    emoji_pattern = re.compile("["
                               "\U0001F600-\U0001F64F"  # emoticons
                               "\U0001F300-\U0001F5FF"  # symbols & pictographs                               
                               "\U0001F680-\U0001F6FF"  # transport & map symbols
                               "\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "\U00002500-\U00002BEF"  # chinese char
                               "\U0001F910-\U0001F970"  # emoticons face
                               "\U00002702-\U000027B0"
                               "\U000024C2-\U0001F251"
                               "\U0001f926-\U0001f937"
                               "\u200d"
                               "\u2640-\u2642"
                               "\U0001F1F2-\U0001F1F4"  # Macau flag
                               "\U0001F1E6-\U0001F1FF"  # flags
                               "\U0001F600-\U0001F64F"
#                                r"\U000263A"              # smiling face
                               "\U0001F1F2"
                               "\U0001F1F4"
                               "\U0001F620"
                               "\U00010000-\U0010ffff"
                               "\u2600-\u2B55"
                               "\u23cf"
                               "\u23e9"
                               "\u231a"
                               "\u3030"
                               "\ufe0f"
                               "]+", flags=re.UNICODE)   
    text = emoji_pattern.sub(r'', text) # no emoji
    return text

def get_dictionary(file):
    dictionary = {}
    with open(file) as file:
         for line in file:
                key_vals = line.split()
                key = key_vals[0]
                v = ''
                for val in key_vals[1:]:
                    v += val+' '
                dictionary[key] = v
    return dictionary

def clean_dictionary(text, dictionary):
    text_clean = text
    for key,value in dictionary.items():
        rex = re.escape(key)
        sub = re.search('(\w+(\s+|[-?!]+))+',value).group()
        text_clean = re.sub(rex,sub,text_clean)
    return text_clean

### Guardamos el corpus ya procesado A

In [5]:
corpus_train_esA = procesar(corpus_train_esA, "public_development_en_TaskA/train_en_clean.tsv")
corpus_dev_esA = procesar(corpus_dev_esA, "public_development_en_TaskA/dev_en_clean.tsv")

In [6]:
#Leyendo el corpus ya procesado

train_idA = corpus_train_esA[corpus_train_esA.columns[0]]
X_train_textA = corpus_train_esA[corpus_train_esA.columns[1]].fillna(' ')
y_train_hsA = corpus_train_esA[corpus_train_esA.columns[2]]

test_idA = corpus_dev_esA[corpus_train_esA.columns[0]]
X_test_textA = corpus_dev_esA[corpus_dev_esA.columns[1]].fillna(' ')
y_test_hsA = corpus_dev_esA[corpus_dev_esA.columns[2]]

In [7]:
# Longitud de train y test

print( len(X_train_textA), len(y_train_hsA) )

print( len(X_test_textA), len(y_test_hsA) )

9000 9000
1000 1000


In [8]:
# Vectorizamos 

cvectorizer = CountVectorizer(
    # lowercase=True,
    #stop_words=[word.decode('utf-8') for word in nltk.corpus.stopwords.words('spanish')],
    #token_pattern=r'\b\w+\b', #selects tokens of 2 or more alphanumeric characters 
    ngram_range=(1,3),#n-grams de palabras n = 1 a n = 3 (unigramas, bigramas y trigramas)
    min_df=5,#ignorando los términos que tienen una frecuencia de documento estrictamente inferior a 5
).fit(X_train_textA)

X_train_cvectorized = cvectorizer.transform(X_train_textA).toarray()
print(X_train_cvectorized.shape)

X_test_cvectorized = cvectorizer.transform(X_test_textA).toarray()
print(X_test_cvectorized.shape)

(9000, 4158)
(1000, 4158)


In [9]:
tvectorizer = TfidfVectorizer(
    # lowercase=True,
    #stop_words=[word.decode('utf-8') for word in nltk.corpus.stopwords.words('spanish')],
    #token_pattern=r'\b\w+\b', #selects tokens of 2 or more alphanumeric characters 
    ngram_range=(1,3),#n-grams de palabras n = 1 a n = 3 (unigramas, bigramas y trigramas)
    min_df=5,#ignorando los términos que tienen una frecuencia de documento estrictamente inferior a 5
).fit(X_train_textA)

X_train_tvectorized = tvectorizer.transform(X_train_textA).toarray()
print(X_train_tvectorized.shape)

X_test_tvectorized = tvectorizer.transform(X_test_textA).toarray()
print(X_test_tvectorized.shape)

(9000, 4158)
(1000, 4158)


# Método 1. Perceptrón Multicapa (Multi-Layer Perceptron, MLP) 

In [10]:
from sklearn.neural_network import MLPClassifier

mlp1 = MLPClassifier(hidden_layer_sizes=(10,10,10), max_iter=500, alpha=0.0001,
                    solver='adam', random_state=21,tol=0.000000001)
mlp2 = MLPClassifier(hidden_layer_sizes=(6,6,6,6),solver='lbfgs',max_iter=6000)

In [11]:

mlp1.fit( X_train_cvectorized, y_train_hsA)
predictions1 = mlp1.predict(X_test_cvectorized)

print('\t', 'Accuracy mlp1 cv', accuracy_score(y_test_hsA, predictions1))
print('\t', 'F1-score mlp1 cv', f1_score(y_test_hsA, predictions1))

######

mlp1.fit( X_train_tvectorized, y_train_hsA)
predictions1 = mlp1.predict(X_test_tvectorized)

print('\t', 'Accuracy mlp1 tfidv', accuracy_score(y_test_hsA, predictions1))
print('\t', 'F1-score mlp1 tfidv', f1_score(y_test_hsA, predictions1))

	 Accuracy mlp1 cv 0.677
	 F1-score mlp1 cv 0.6291618828932263
	 Accuracy mlp1 tfidv 0.664
	 F1-score mlp1 tfidv 0.619047619047619


In [12]:

mlp2.fit( X_train_cvectorized, y_train_hsA)
predictions2 = mlp2.predict(X_test_cvectorized)

print('\t', 'Accuracy mlp2 cv', accuracy_score(y_test_hsA, predictions2))
print('\t', 'F1-score mlp2 cv', f1_score(y_test_hsA, predictions2))

######

mlp2.fit( X_train_tvectorized, y_train_hsA)
predictions2 = mlp2.predict(X_test_tvectorized)

print('\t', 'Accuracy mlp2 tfidv', accuracy_score(y_test_hsA, predictions2))
print('\t', 'F1-score mlp2 tfidv', f1_score(y_test_hsA, predictions2))

	 Accuracy mlp2 cv 0.704
	 F1-score mlp2 cv 0.6782608695652174
	 Accuracy mlp2 tfidv 0.573
	 F1-score mlp2 tfidv 0.0


In [13]:
# normalizacion de los datos
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train_cvectorized)
X_test = scaler.fit_transform(X_test_cvectorized)

In [14]:
mlp2.fit( X_train, y_train_hsA)
predictions2 = mlp2.predict(X_test)

print('\t', 'Accuracy mlp2', accuracy_score(y_test_hsA, predictions2))
print('\t', 'F1-score mlp2', f1_score(y_test_hsA, predictions2))

	 Accuracy mlp2 0.674
	 F1-score mlp2 0.6393805309734514


No hay diferencia significativa entre normalizar o no los datos

# Método 2 Redes Neuronales Recurrentes 

In [16]:
print( X_train_cvectorized.shape, len(y_train_hsA), 'Secuencia de entrenamiento' )

print( X_test_cvectorized.shape, len(y_test_hsA), 'Secuencia de prueba' )

(9000, 4158) 9000 Secuencia de entrenamiento
(1000, 4158) 1000 Secuencia de prueba


In [17]:
from keras.models import Sequential
model = Sequential()

In [18]:
# La clase layer de redes RNN
from keras.layers import Embedding, SimpleRNN

# Como cualquier otra layer de Keras, SimpleRNN procesa lotes de secuencias Numpy.
# La entrada es de la forma (batch_size, timesteps, input_features) en vez de (timesteps, input_features).
# [muestras, pasos de tiempo, características]

from keras.layers import Dense

max_features = 10000  # tamaño del diccionario de palabras comunes
                      # (número de palabras a utilizar)
maxlen = 1775         # longitud máxima de cada secuencia 
batch_size = 32

# Capa embedding
# input_dim : tamaño del vocabulario
# output_dim: dimensión del vector al que se mapea
#Se usa un embedding con tamaño de diccionario a los más de 10,000 y se mapean a dimensión un vector de dimensión 32
model.add(Embedding(input_dim=max_features, output_dim=32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))
# RNN con una capa Embedding y una capa SimpleRNN que regresa solo una salida para cada secuencia

# Resumen de la arquitectura
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          320000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                2080      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 322,113
Trainable params: 322,113
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['acc']
)

import time
tic = time.time()
history = model.fit(
    X_train_cvectorized, y_train_hsA,
    epochs=10,
    batch_size=128,
    validation_split=0.2,
    verbose=2
)
print('Tiempo de entrenamiento:', time.time()-tic)

Epoch 1/10
57/57 - 101s - loss: 0.6954 - acc: 0.5064 - val_loss: 0.6882 - val_acc: 0.7111 - 101s/epoch - 2s/step
Epoch 2/10
57/57 - 99s - loss: 0.6935 - acc: 0.5194 - val_loss: 0.6508 - val_acc: 0.8611 - 99s/epoch - 2s/step
Epoch 3/10
57/57 - 100s - loss: 0.6954 - acc: 0.4989 - val_loss: 0.6496 - val_acc: 0.8617 - 100s/epoch - 2s/step
Epoch 4/10
57/57 - 101s - loss: 0.6927 - acc: 0.5099 - val_loss: 0.7116 - val_acc: 0.1633 - 101s/epoch - 2s/step
Epoch 5/10
57/57 - 99s - loss: 0.6919 - acc: 0.5067 - val_loss: 0.6967 - val_acc: 0.1706 - 99s/epoch - 2s/step
Epoch 6/10
57/57 - 99s - loss: 0.6934 - acc: 0.4981 - val_loss: 0.7426 - val_acc: 0.1389 - 99s/epoch - 2s/step
Epoch 7/10
57/57 - 99s - loss: 0.6980 - acc: 0.4953 - val_loss: 0.5661 - val_acc: 0.8244 - 99s/epoch - 2s/step
Epoch 8/10
57/57 - 101s - loss: 0.7086 - acc: 0.5021 - val_loss: 0.7567 - val_acc: 0.2067 - 101s/epoch - 2s/step
Epoch 9/10
57/57 - 101s - loss: 0.6956 - acc: 0.5079 - val_loss: 0.6768 - val_acc: 0.7628 - 101s/epoch -

In [21]:
# evaluate the model
scores = model.evaluate(X_test_cvectorized, y_test_hsA, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
# La función model.evaluate predice la salida para la entrada dada y luego calcula la función de métrica 
# especificada en model.compile y basada en y_true y y_pred y devuelve el valor de métrica calculada como salida

# make predictions
testPredict = model.predict(X_test_cvectorized)
# model.predict simplemente devuelve el y_pred
print('\t', 'Accuracy', accuracy_score(y_test_hsA, testPredict.round()))

# si usamos model.predict y luego calculamos las métricas uno mismo, el valor de la métrica calculada 
# debería resultar ser el mismo que model.evaluate

acc: 46.20%
	 Accuracy 0.462


# Método 3. Gradient Boosting Classifier

In [22]:
from sklearn.ensemble import GradientBoostingClassifier

In [23]:
gb = GradientBoostingClassifier(random_state=42, loss='exponential', learning_rate= 0.1, n_estimators=100)

In [24]:
gb.fit(X_train_cvectorized,y_train_hsA)

GradientBoostingClassifier(loss='exponential', random_state=42)

In [25]:
gb.score(X_test_cvectorized, y_test_hsA)

0.713

De los 3 métodos que probamos el Gradient Boosting Classifier (GBC) es el mejor ya que no sólo es el que mejor score tiene, sino el que menos costo computacional requiere.